In [34]:
from sqlalchemy import create_engine, text
import pandas as pd
import psycopg2
import json

In [29]:
# Cargar credenciales desde el archivo JSON
with open("../credenciales.json", "r") as file:
    credenciales = json.load(file)

# Establecer la conexión
try:
    conn = psycopg2.connect(
        user=credenciales["user"],
        password=credenciales["password"],
        host=credenciales["host"],
        port=credenciales["port"],
        database=credenciales["database"]
    )
    print("¡Conexión exitosa!")
except Exception as e:
    print(f"Error al conectar a la base de datos: {e}")
finally:
    if conn:
        conn.close()

¡Conexión exitosa!


In [30]:
# Leer el archivo CSV
df = pd.read_csv("../data/raw/candidates.csv")

# Mostrar las primeras filas
print(df.head())

total_filas_csv = len(df)
print("")
print(f"El archivo CSV tiene un total de {total_filas_csv} filas.")

  First Name;Last Name;Email;Application Date;Country;YOE;Seniority;Technology;Code Challenge Score;Technical Interview Score
0  Bernadette;Langworth;leonard91@yahoo.com;2021-...                                                                         
1  Camryn;Reynolds;zelda56@hotmail.com;2021-09-09...                                                                         
2  Larue;Spinka;okey_schultz41@gmail.com;2020-04-...                                                                         
3  Arch;Spinka;elvera_kulas@yahoo.com;2020-10-01;...                                                                         
4  Larue;Altenwerth;minnie.gislason@gmail.com;202...                                                                         

El archivo CSV tiene un total de 50000 filas.


In [ ]:
# Cargar credenciales desde el archivo JSON
credentials = "../credenciales.json"
with open(credentials) as f:
    creds = json.load(f)

# Credenciales de la base de datos
DB_USER = creds["user"]
DB_PASSWORD = creds["password"]
DB_HOST = creds["host"]
DB_PORT = creds["port"]
DB_NAME = creds["database"]

# Crear Base de Datos si no existe
creator_connection_string = f'postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/postgres'
creator_engine = create_engine(creator_connection_string)

with creator_engine.connect() as connection:
    connection.execution_options(isolation_level="AUTOCOMMIT")
    result = connection.execute(
        text(f"SELECT 1 FROM pg_database WHERE datname = '{DB_NAME}'")
    )
    
    if not result.scalar():
        connection.execute(text(f"CREATE DATABASE {DB_NAME}"))
        print(f"Base de datos '{DB_NAME}' creada exitosamente!")
    else:
        print(f"La base de datos '{DB_NAME}' ya existe.")

# Crear conexión a la base de datos ya creada
connection_string = f'postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}'
engine = create_engine(connection_string)

# Cargar el DataFrame desde el archivo CSV
df = pd.read_csv("../data/raw/candidates.csv")

# Crear la tabla en la base de datos y cargar los datos
table_name = "candidates"  # Nombre de la tabla
df.to_sql(table_name, engine, if_exists='replace', index=False)

print(f"Tabla '{table_name}' creada y datos cargados en la base de datos '{DB_NAME}'.")

La base de datos 'candidates' ya existe.
Tabla 'candidates' creada y datos cargados en la base de datos 'candidates'.


In [32]:
# Conectar a la base de datos
connection_string = f'postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}'
engine = create_engine(connection_string)

# Verificar los datos en la nueva tabla
with engine.connect() as connection:
    result = connection.execute(text(f"SELECT * FROM {table_name} LIMIT 5;"))
    print(f"Primeros 5 registros de la tabla '{table_name}':")
    for row in result:
        print(row)

Primeros 5 registros de la tabla 'candidates':
('Bernadette;Langworth;leonard91@yahoo.com;2021-02-26;Norway;2;Intern;Data Engineer;3;3',)
('Camryn;Reynolds;zelda56@hotmail.com;2021-09-09;Panama;10;Intern;Data Engineer;2;10',)
('Larue;Spinka;okey_schultz41@gmail.com;2020-04-14;Belarus;4;Mid-Level;Client Success;10;9',)
('Arch;Spinka;elvera_kulas@yahoo.com;2020-10-01;Eritrea;25;Trainee;QA Manual;7;1',)
('Larue;Altenwerth;minnie.gislason@gmail.com;2020-05-20;Myanmar;13;Mid-Level;Social Media Community Management;9;7',)
